<a href="https://colab.research.google.com/github/TheMagicShop/Fourier-Series-illustration/blob/main/Fourier_Animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script helps you visualize the fourier series converging(or actually not) to the true signal, or either to have an insight of the gibbs phenomenon.

The fourier series are valid for a broad class of signals that have a physical sense, maybe the most practical way is checking for Dirichlet conditions, but since no one seeing you, you can apply this to any function you want.

In [54]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import warnings
warnings.filterwarnings(action="ignore")
%matplotlib inline
################################################################################
def fourier_coef(func , period , n , alpha=0 , even=False , odd=False):
  # compute the n-th fourier cofficient (an,bn)
  # if your function is either even or odd please mention it
  # if you are constructiong your own function in a certain interval
  # you can specify alpha and make your life easier
  # for example a function is defined in (-1,4), you can put alpha=-1 and T=5
  T = period
  omega = 2*np.pi / T
  round_err = 12

  if n == 0:  ## compute a0
    if odd:
      return 0
    else:
      a0, _ = quad(func , alpha , alpha+T)
      a0 = a0 / T
      a0 = round(a0 , round_err)
      return (a0 , 0)

  an , bn = 0 , 0
  if not odd:
    an , _ = quad(lambda t : func(t) * np.cos(n*omega*t) , alpha , alpha+T)
    an = 2 * an / T
    an = round(an , round_err)
  if not even:
    bn , _ = quad(lambda t : func(t) * np.sin(n*omega*t) , alpha , alpha+T)
    bn = 2 * bn / T
    bn = round(bn , round_err)  
  return (an , bn)
################################################################################
def fourier_serie(func , period , N , alpha=0 , even=False , odd=False):
  # compute the fourier serie till the N-th term
  T = period
  omega = 2*np.pi / T
  
  line = np.linspace(-2*T , 2*T , 1000) # x values

  def term(n):    # an*cos(nwx) + bn*sin(nwx)   -for all x in (-2T,2T)
    an , bn = fourier_coef(func , period , n , alpha=alpha , even=even , odd=odd)
    return an * np.cos(n * omega * line) + bn * np.sin(n * omega * line)
  
  SN = np.empty((line.shape[0] , N+1))
  SN[:,0] = term(0).T
  for n in range(1,N+1):
    SN[:,n] = term(n).T + SN[:,n-1]

  return SN
################################################################################
def plot_fourier(func , period , N , alpha=0 , show_original=True , even=False , odd=False):
  # return an animation of the fourie serie from 0(dc) to N
  # if you dont want to show the original signal 
  # in the background please mention show_original=False
  T = period
  SN = fourier_serie(func , period , N , alpha=alpha , even=even , odd=odd)

  line = np.linspace(-2*T , 2*T , 1000)

  ymin , ymax = SN.min() , SN.max()
  fig = plt.figure(figsize=(12,6))
  ax = fig.add_axes([0.03 , 0.07 , 0.95 , 0.88])
  ax.set_xlim((-2*T , 2*T))
  ax.set_ylim((0.9*ymin , 1.1*ymax))
  ax.set_xticks([-T,0,T])
  ax.set_xticklabels(["-T" , "0" , "T"] , fontsize=14)
  ax.set_title("Fourier Serie" , fontdict={'color':'midnightblue' , 'fontsize':'xx-large'})
  p, = ax.plot([], [], lw=2)

  if show_original:
    l = np.linspace(alpha,alpha+T , 250)
    vz = np.vectorize(func)
    out = vz(l)
    L = np.hstack((l-3*T , l-2*T , l-T , l , l+T , l+2*T , l+3*T))
    OUT = np.hstack((out , out , out , out , out , out , out))
    ax.plot(L , OUT , "--g" , alpha = 0.5)


  def init():
      p.set_data([], [])
      return (p,)

  def animate(n):
    p.set_data(line , SN[:,n])
    p.set_label(str(n))
    ax.legend(loc="upper right", title='N' ,title_fontsize = 16, prop={'size': 16})
    return (p,)

  if N < 5:
    interval = 2000
  elif N < 10:
    interval = 1000
  elif N < 25:
    interval = 800
  elif N < 50:
    interval = 500
  else:
    interval = 200
  
  anim = animation.FuncAnimation(fig, animate, init_func=init,
                                frames=N+1, interval=interval, blit=True)
  
  X = HTML(anim.to_html5_video())

  fig.delaxes(ax)
  return X

if your signal is either odd or even please mention it by setting `odd=True` or `even=True`, if your making your own periodic signal from an aperiodic signal in some interval [a,a+T] you can set `alpha=a`, the original signal will show by default in the background if you don't like it please set `show_original=True `.

so just call the function:


```
plot_fourier(func , period , N , alpha=0 , show_original=True , even=False , odd=False)
```
or equivalently 


```
plot_fourier(func , period , N)
```

where `alpha`, `show_original`, `even` and `odd` are set to default values.

`func` is the signal, and `N` is the end term in your fourier serie



# Let's try some Examples

In [ ]:
# Example 1
# this a 1-periodic function equals 1 in [0,1/2] and 0 in ]1/2,1] 

func = lambda x : 1 if x<=1/2 else 0
period = 1
N = 180

plot_fourier(func, period, N) #call

In [ ]:
# Example 2
# trying to aproach the pulse train

func = lambda x : 1 if x<=1/100 else 0
period = 1
N = 150

plot_fourier(func, period, N) #call

In [ ]:
# Example 3
# this is the funcion that equals x in [-1/2,1/2] and repeated with a period 1

func = lambda x : x 
period = 1
alpha = -1/2
N = 100

plot_fourier(func, period, N, alpha=alpha)

In [ ]:
# Example 4

func = lambda x : np.sin(np.tan(x))
period = np.pi/2
N=100

plot_fourier(func, period, N)

In [ ]:
# Example 5

func = lambda x : np.sin(1/np.sin(x))
period = 2*np.pi
N = 200

plot_fourier(func, period, N)